In [1]:
import pandas as pd
import numpy as np
import math
import warnings
import re

In [2]:
warnings.filterwarnings("ignore")

## Read Data

In [3]:
df = pd.read_csv("Global YouTube Statistics.csv", encoding="unicode_escape")
df.head()

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
0,1,T-Series,245000000,2.280000e+11,Music,T-Series,20082,India,IN,Music,...,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880
1,2,YouTube Movies,170000000,0.000000e+00,Film & Animation,youtubemovies,1,United States,US,Games,...,NaN,2006.0,Mar,5.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
2,3,MrBeast,166000000,2.836884e+10,Entertainment,MrBeast,741,United States,US,Entertainment,...,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
3,4,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,Cocomelon - Nursery Rhymes,966,United States,US,Education,...,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
4,5,SET India,159000000,1.480000e+11,Shows,SET India,116536,India,IN,Entertainment,...,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880


In [4]:
df.shape

(995, 28)

In [5]:
df[['Title', 'Youtuber']]

,Title,Youtuber
0,T-Series,T-Series
1,youtubemovies,YouTube Movies
2,MrBeast,MrBeast
3,Cocomelon - Nursery Rhymes,Cocomelon - Nursery Rhymes
4,SET India,SET India
...,...,...
990,Natan por Aï¿,Natan por Aï¿
991,Free Fire India Official,Free Fire India Official
992,HybridPanda,Panda
993,RobTopGames,RobTopGames


In [6]:
# Drop unnecessary columns
dropped_cols = ['Title', 'Abbreviation', 'Unemployment rate', 'Latitude', 'Longitude', 'rank', 'video_views_rank', 'country_rank', 'channel_type_rank', 'channel_type']
df1 = df.drop(dropped_cols, axis=1)
df1.head()

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
0,T-Series,245000000,2.280000e+11,Music,20082,India,2.258000e+09,564600.0,9000000.00,6800000.00,1.084000e+08,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,471031528.0
1,YouTube Movies,170000000,0.000000e+00,Film & Animation,1,United States,1.200000e+01,0.0,0.05,0.04,5.800000e-01,NaN,2006.0,Mar,5.0,88.2,3.282395e+08,270663028.0
2,MrBeast,166000000,2.836884e+10,Entertainment,741,United States,1.348000e+09,337000.0,5400000.00,4000000.00,6.470000e+07,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,270663028.0
3,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,966,United States,1.975000e+09,493800.0,7900000.00,5900000.00,9.480000e+07,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,270663028.0
4,SET India,159000000,1.480000e+11,Shows,116536,India,1.824000e+09,455900.0,7300000.00,5500000.00,8.750000e+07,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,471031528.0


In [7]:
df1.describe()

,subscribers,video views,uploads,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_date,Gross tertiary education enrollment (%),Population,Urban_population
count,9.950000e+02,9.950000e+02,995.000000,9.390000e+02,995.000000,9.950000e+02,9.950000e+02,9.950000e+02,6.580000e+02,990.000000,990.000000,872.000000,8.720000e+02,8.720000e+02
mean,2.298241e+07,1.103954e+10,9187.125628,1.756103e+08,36886.148281,5.898078e+05,4.422574e+05,7.081814e+06,3.490791e+05,2012.630303,15.746465,63.627752,4.303873e+08,2.242150e+08
std,1.752611e+07,1.411084e+10,34151.352254,4.163782e+08,71858.724092,1.148622e+06,8.612161e+05,1.379704e+07,6.143554e+05,4.512503,8.777520,26.106893,4.727947e+08,1.546874e+08
min,1.230000e+07,0.000000e+00,0.000000,1.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1970.000000,1.000000,7.600000,2.025060e+05,3.558800e+04
25%,1.450000e+07,4.288145e+09,194.500000,2.013750e+07,2700.000000,4.350000e+04,3.265000e+04,5.217500e+05,1.000000e+05,2009.000000,8.000000,36.300000,8.335541e+07,5.590832e+07
50%,1.770000e+07,7.760820e+09,729.000000,6.408500e+07,13300.000000,2.127000e+05,1.595000e+05,2.600000e+06,2.000000e+05,2013.000000,16.000000,68.000000,3.282395e+08,2.706630e+08
75%,2.460000e+07,1.355470e+10,2667.500000,1.688265e+08,37900.000000,6.068000e+05,4.551000e+05,7.300000e+06,4.000000e+05,2016.000000,23.000000,88.200000,3.282395e+08,2.706630e+08
max,2.450000e+08,2.280000e+11,301308.000000,6.589000e+09,850900.000000,1.360000e+07,1.020000e+07,1.634000e+08,8.000000e+06,2022.000000,31.000000,113.100000,1.397715e+09,8.429340e+08


In [8]:
df2 = df1[(df1['video views']!=0) & (df1['uploads']>100)]
df2.describe()

,subscribers,video views,uploads,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_date,Gross tertiary education enrollment (%),Population,Urban_population
count,8.150000e+02,8.150000e+02,815.000000,8.150000e+02,815.000000,8.150000e+02,8.150000e+02,8.150000e+02,5.740000e+02,815.000000,815.000000,774.000000,7.740000e+02,7.740000e+02
mean,2.267681e+07,1.156397e+10,11211.206135,1.859554e+08,43898.589387,7.019649e+05,5.263405e+05,8.427945e+06,3.917884e+05,2012.602454,15.737423,62.696382,4.485580e+08,2.290174e+08
std,1.729635e+07,1.515754e+10,37437.248009,3.734304e+08,77394.237978,1.237101e+06,9.275460e+05,1.485987e+07,6.460873e+05,4.350887,8.756872,26.129059,4.853834e+08,1.568250e+08
min,1.230000e+07,2.056338e+07,104.000000,8.600000e+01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.200000e+01,1970.000000,1.000000,9.000000,1.912789e+06,1.304943e+06
25%,1.450000e+07,4.350995e+09,459.500000,2.863300e+07,6700.000000,1.077000e+05,8.075000e+04,1.300000e+06,1.000000e+05,2009.000000,8.000000,36.000000,9.646211e+07,5.590832e+07
50%,1.760000e+07,7.876741e+09,1101.000000,8.006200e+07,19300.000000,3.082000e+05,2.312000e+05,3.700000e+06,2.000000e+05,2013.000000,16.000000,63.200000,3.282395e+08,2.706630e+08
75%,2.410000e+07,1.399023e+10,3730.000000,1.833710e+08,45150.000000,7.227000e+05,5.420000e+05,8.700000e+06,4.000000e+05,2016.000000,23.000000,88.200000,3.282395e+08,2.706630e+08
max,2.450000e+08,2.280000e+11,301308.000000,6.148000e+09,850900.000000,1.360000e+07,1.020000e+07,1.634000e+08,8.000000e+06,2022.000000,31.000000,113.100000,1.397715e+09,8.429340e+08


In [9]:
df2.head()

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
0,T-Series,245000000,2.280000e+11,Music,20082,India,2.258000e+09,564600.0,9000000.0,6800000.0,108400000.0,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,471031528.0
2,MrBeast,166000000,2.836884e+10,Entertainment,741,United States,1.348000e+09,337000.0,5400000.0,4000000.0,64700000.0,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,270663028.0
3,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,966,United States,1.975000e+09,493800.0,7900000.0,5900000.0,94800000.0,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,270663028.0
4,SET India,159000000,1.480000e+11,Shows,116536,India,1.824000e+09,455900.0,7300000.0,5500000.0,87500000.0,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,471031528.0
6,ýýý Kids Diana Show,112000000,9.324704e+10,People & Blogs,1111,United States,7.316740e+08,182900.0,2900000.0,2200000.0,35100000.0,NaN,2015.0,May,12.0,88.2,3.282395e+08,270663028.0


In [10]:
df2.shape

(815, 18)

In [11]:
df2.isnull().sum()

Youtuber                                     0
subscribers                                  0
video views                                  0
category                                    35
uploads                                      0
Country                                     40
video_views_for_the_last_30_days             0
lowest_monthly_earnings                      0
highest_monthly_earnings                     0
lowest_yearly_earnings                       0
highest_yearly_earnings                      0
subscribers_for_last_30_days               241
created_year                                 0
created_month                                0
created_date                                 0
Gross tertiary education enrollment (%)     41
Population                                  41
Urban_population                            41
dtype: int64

In [12]:
# null to data by cells
total_null_cells = df2.isnull().sum().sum()
total_cells = np.product(df2.shape)
total_null_cells*100/total_cells

2.992501704158146

In [13]:
# null to data by rows
total_null_rows = len(df2[df2.isnull().any(axis=1)])
total_rows = df2.shape[0]
total_null_rows*100/total_rows

37.54601226993865

In [14]:
df2.category.unique()

array(['Music', 'Entertainment', 'Education', 'Shows', 'People & Blogs',
       'Gaming', 'Sports', 'Film & Animation', 'News & Politics',
       'Comedy', 'Trailers', 'Nonprofits & Activism', nan,
       'Howto & Style', 'Science & Technology', 'Movies',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events'],
      dtype=object)

## Handle missing values

### category

In [15]:
df2[pd.isna(df2.category) == True]

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
91,Vlad vï¿½ï¿½ï,37900000,2.351015e+10,NaN,515,United States,2.440930e+08,61000.0,976400.0,732300.0,11700000.0,200000.0,2018.0,Jul,20.0,88.2,3.282395e+08,270663028.0
132,BETER Bï¿½ï¿½,32700000,1.918004e+10,NaN,2738,Turkey,1.463000e+09,365900.0,5900000.0,4400000.0,70200000.0,2100000.0,2012.0,Dec,12.0,23.9,8.342962e+07,63097818.0
155,Diana and Roma ESP,30700000,1.235599e+10,NaN,578,United States,1.029980e+08,25700.0,412000.0,309000.0,4900000.0,200000.0,2016.0,Nov,27.0,88.2,3.282395e+08,270663028.0
161,ýýýýýýýýýýýýýýý ýýýýýý ýýýýýýýýýýýýýýýýýý,30400000,1.799996e+10,NaN,532,United States,1.229260e+08,30700.0,491700.0,368800.0,5900000.0,300000.0,2018.0,Nov,3.0,88.2,3.282395e+08,270663028.0
214,D Billions,26200000,3.197746e+10,NaN,775,United States,4.870760e+08,121800.0,1900000.0,1500000.0,23400000.0,200000.0,2019.0,Jul,27.0,88.2,3.282395e+08,270663028.0
255,shfa show India,24200000,1.572416e+10,NaN,469,United Arab Emirates,1.512080e+08,37800.0,604800.0,453600.0,7300000.0,200000.0,2019.0,Apr,1.0,36.8,9.770529e+06,8479744.0
261,Diana and Roma ARA,24100000,1.104126e+10,NaN,590,United States,6.688400e+07,16700.0,267500.0,200700.0,3200000.0,200000.0,2019.0,Feb,1.0,88.2,3.282395e+08,270663028.0
312,Vlad and Niki Arabic,22500000,1.061370e+10,NaN,526,United States,6.506600e+07,16300.0,260300.0,195200.0,3100000.0,100000.0,2018.0,Nov,9.0,88.2,3.282395e+08,270663028.0
319,Vlad and Niki ESP,22000000,8.594441e+09,NaN,515,United States,1.838710e+08,46000.0,735500.0,551600.0,8800000.0,200000.0,2018.0,Oct,11.0,88.2,3.282395e+08,270663028.0
332,ZAMZAM ELECTRONICS TRADING,21700000,8.507819e+09,NaN,15126,United Arab Emirates,1.598000e+09,399600.0,6400000.0,4800000.0,76700000.0,3400000.0,2020.0,Dec,19.0,36.8,9.770529e+06,8479744.0


In [16]:
# In this case, we can remove these rows but we try to avoid such solution as much as possible
# Another option is to assign 'Other' as a new category to these null values
df2.category = df2.category.apply(lambda category: 'Other' if pd.isna(category) == True else category)
df2.category.unique()

array(['Music', 'Entertainment', 'Education', 'Shows', 'People & Blogs',
       'Gaming', 'Sports', 'Film & Animation', 'News & Politics',
       'Comedy', 'Trailers', 'Nonprofits & Activism', 'Other',
       'Howto & Style', 'Science & Technology', 'Movies',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events'],
      dtype=object)

In [17]:
df2.category.isnull().sum()

0

### country

In [18]:
df2[pd.isna(df2.Country) == True]

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
74,elrubiusOMG,40400000,7.410537e+09,Gaming,703,NaN,5.773000e+06,1400.0,23100.0,17300.0,277100.0,NaN,2011.0,Dec,20.0,NaN,NaN,NaN
100,PowerKids TV,36500000,1.896124e+10,Film & Animation,5621,NaN,3.187020e+08,79700.0,1300000.0,956100.0,15300000.0,600000.0,2014.0,Aug,6.0,NaN,NaN,NaN
103,Ozuna,36200000,2.335580e+10,Music,196,NaN,1.692420e+08,42300.0,677000.0,507700.0,8100000.0,100000.0,2016.0,Mar,24.0,NaN,NaN,NaN
140,SMTOWN,31900000,2.733024e+10,Music,4158,NaN,2.334020e+08,58400.0,933600.0,700200.0,11200000.0,100000.0,2006.0,Mar,18.0,NaN,NaN,NaN
169,jacksepticeye,30100000,1.624663e+10,Gaming,5114,NaN,4.042500e+07,10100.0,161700.0,121300.0,1900000.0,100000.0,2007.0,Feb,24.0,NaN,NaN,NaN
189,Goldmines Dishoom,28000000,8.603468e+09,Film & Animation,3878,NaN,1.292110e+08,32300.0,516800.0,387600.0,6200000.0,400000.0,2016.0,Nov,26.0,NaN,NaN,NaN
202,Gulshan Kalra,27100000,1.273244e+10,Howto & Style,545,NaN,5.027790e+08,125700.0,2000000.0,1500000.0,24100000.0,1300000.0,2018.0,Apr,9.0,NaN,NaN,NaN
218,NichLmao,26000000,1.410101e+10,People & Blogs,868,NaN,6.063620e+08,151600.0,2400000.0,1800000.0,29100000.0,1100000.0,2018.0,Sep,25.0,NaN,NaN,NaN
245,mujjuu___14,24800000,1.465553e+10,People & Blogs,872,NaN,4.002220e+08,100100.0,1600000.0,1200000.0,19200000.0,900000.0,2020.0,May,18.0,NaN,NaN,NaN
258,Daniel LaBelle,24100000,1.291616e+10,Comedy,287,NaN,3.987650e+08,99700.0,1600000.0,1200000.0,19100000.0,700000.0,2009.0,Mar,11.0,NaN,NaN,NaN


In [19]:
# We cannot estimate country because it's a categorical field, so the best way is to remove all rows that have unknown country
df3 = df2[~(pd.isna(df2.Country) == True)]
df3.shape

(775, 18)

In [20]:
df3.isnull().sum()

Youtuber                                     0
subscribers                                  0
video views                                  0
category                                     0
uploads                                      0
Country                                      0
video_views_for_the_last_30_days             0
lowest_monthly_earnings                      0
highest_monthly_earnings                     0
lowest_yearly_earnings                       0
highest_yearly_earnings                      0
subscribers_for_last_30_days               234
created_year                                 0
created_month                                0
created_date                                 0
Gross tertiary education enrollment (%)      1
Population                                   1
Urban_population                             1
dtype: int64

### Education, population & urban population

In [21]:
df3[pd.isna(df3.Urban_population) == True]

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
663,Musas,15100000,2.400038e+09,Howto & Style,159,Andorra,86.0,0.02,0.34,0.26,4.0,NaN,2006.0,Mar,12.0,NaN,NaN,NaN


In [22]:
df3[df3.Country == 'Andorra']

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
663,Musas,15100000,2.400038e+09,Howto & Style,159,Andorra,86.0,0.02,0.34,0.26,4.0,NaN,2006.0,Mar,12.0,NaN,NaN,NaN


In [23]:
# There is only one row so we can just remove it
df4 = df3[~(df3.Country == 'Andorra')]
df4.shape

(774, 18)

### subscribers for last 30 days

In [24]:
df4[pd.isna(df4.subscribers_for_last_30_days) == True]

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
6,ýýý Kids Diana Show,112000000,9.324704e+10,People & Blogs,1111,United States,731674000.0,182900.0,2900000.0,2200000.0,35100000.0,NaN,2015.0,May,12.0,88.2,3.282395e+08,270663028.0
7,PewDiePie,111000000,2.905804e+10,Gaming,4716,Japan,39184000.0,9800.0,156700.0,117600.0,1900000.0,NaN,2010.0,Apr,29.0,63.2,1.262266e+08,115782416.0
23,Canal KondZilla,66500000,3.677559e+10,Music,2572,Brazil,447223000.0,0.0,0.0,0.0,0.0,NaN,2012.0,Mar,21.0,51.3,2.125594e+08,183241641.0
59,whinderssonnunes,44200000,4.274709e+09,Comedy,558,Brazil,26683000.0,6700.0,106700.0,80100.0,1300000.0,NaN,2013.0,Jan,21.0,51.3,2.125594e+08,183241641.0
68,Like Nastya Show,42400000,1.954770e+10,Entertainment,618,United States,53202000.0,13300.0,212800.0,159600.0,2600000.0,NaN,2017.0,Nov,14.0,88.2,3.282395e+08,270663028.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,SAAIHALILINTAR,12400000,1.113066e+09,Entertainment,409,Indonesia,806075.0,202.0,3200.0,2400.0,38700.0,NaN,2012.0,Apr,28.0,36.3,2.702039e+08,151509724.0
978,Heidi and Zidane HZHtube,12400000,4.021409e+09,Entertainment,813,Canada,9595000.0,2400.0,38400.0,28800.0,460600.0,NaN,2015.0,Nov,24.0,68.9,3.699198e+07,30628482.0
981,Pari's Lifestyle,12400000,7.741765e+09,People & Blogs,459,India,38165000.0,9500.0,152700.0,114500.0,1800000.0,NaN,2018.0,Nov,26.0,28.1,1.366418e+09,471031528.0
982,DisneyChannelUK,12400000,1.260749e+10,Music,4422,United Kingdom,34758000.0,8700.0,139000.0,104300.0,1700000.0,NaN,2007.0,Dec,6.0,60.0,6.683440e+07,55908316.0


In [25]:
df4.subscribers_for_last_30_days.unique()

array([2.0e+06, 8.0e+06, 1.0e+06,     nan, 1.0e+05, 6.0e+05, 1.1e+06,
       7.0e+05, 4.0e+05, 9.0e+05, 5.0e+05, 3.0e+05, 1.6e+06, 8.0e+05,
       2.0e+05, 1.3e+06, 2.0e+03, 5.5e+06, 1.9e+06, 4.0e+04, 2.1e+06,
       1.2e+06, 1.4e+06, 3.4e+06, 6.7e+06, 3.2e+06, 2.3e+05, 1.0e+03,
       3.0e+06, 8.0e+02, 1.7e+06, 6.0e+02, 1.0e+04, 1.5e+06, 3.2e+01])

In [26]:
# For this field, we are also unable to estimate the number of new subscribers
# Therefore, it would be just a temporary solution to assume there were no new subscribers to these youtube channels
df4.subscribers_for_last_30_days = df4.subscribers_for_last_30_days.apply(lambda new_subscribers: 0 if pd.isna(new_subscribers) == True else new_subscribers)
df4[pd.isna(df4.subscribers_for_last_30_days) == True]

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population


In [27]:
df4.isnull().sum()

Youtuber                                   0
subscribers                                0
video views                                0
category                                   0
uploads                                    0
Country                                    0
video_views_for_the_last_30_days           0
lowest_monthly_earnings                    0
highest_monthly_earnings                   0
lowest_yearly_earnings                     0
highest_yearly_earnings                    0
subscribers_for_last_30_days               0
created_year                               0
created_month                              0
created_date                               0
Gross tertiary education enrollment (%)    0
Population                                 0
Urban_population                           0
dtype: int64

In [28]:
df4.shape

(774, 18)

## Handle special characters

In [29]:
df4.Youtuber.unique()

array(['T-Series', 'MrBeast', 'Cocomelon - Nursery Rhymes', 'SET India',
       'ýýý Kids Diana Show', 'PewDiePie', 'Like Nastya', 'Vlad and Niki',
       'Zee Music Company', 'WWE', 'BLACKPINK', 'Sony SAB', 'BANGTANTV',
       'Justin Bieber', 'HYBE LABELS', 'Zee TV',
       "Pinkfong Baby Shark - Kids' Songs & Stories", 'Canal KondZilla',
       'ChuChu TV Nursery Rhymes & Kids Songs', 'Shemaroo Filmi Gaane',
       'Colors TV', 'Dude Perfect', 'Movieclips', 'Tips Official',
       'El Reino Infantil', 'Wave Music', 'Aaj Tak', 'Sony Music India',
       'EminemMusic', 'Marshmello', 'YRF', 'Ed Sheeran',
       'Infobells - Hindi', 'Taylor Swift', 'Ariana Grande',
       'BillionSurpriseToys  - Nursery Rhymes & Cartoons', 'JuegaGerman',
       'Get Movies', 'Shemaroo', 'SonyMusicIndiaVEVO', 'Fernanfloo',
       'Voot Kids', 'Felipe Neto', 'Vocï¿½ï¿½ï¿½', 'HAR PAL GEO',
       'Katy Perry', 'whinderssonnunes', 'ABS-CBN Entertainment',
       'Shakira', 'ýýýýýýýý ýý ýýýýýýýýýýýýýý', 'ARY

In [30]:
len(df4.Youtuber.unique())

774

In [31]:
# There are names that contain special characters, we will remove all those data
words_with_special_chars = []
for word in df4.Youtuber.unique():
    if 'ï' in word or 'ý' in word:
        words_with_special_chars.append(word)
len(words_with_special_chars)

87

In [32]:
df4.Youtuber = df4.Youtuber.apply(lambda name: 'Dropped' if ('ï' in name or 'ý' in name) else name)
len(df4.Youtuber.unique())

688

In [33]:
df5 = df4[df4.Youtuber != 'Dropped']
df5.Youtuber.unique()

array(['T-Series', 'MrBeast', 'Cocomelon - Nursery Rhymes', 'SET India',
       'PewDiePie', 'Like Nastya', 'Vlad and Niki', 'Zee Music Company',
       'WWE', 'BLACKPINK', 'Sony SAB', 'BANGTANTV', 'Justin Bieber',
       'HYBE LABELS', 'Zee TV',
       "Pinkfong Baby Shark - Kids' Songs & Stories", 'Canal KondZilla',
       'ChuChu TV Nursery Rhymes & Kids Songs', 'Shemaroo Filmi Gaane',
       'Colors TV', 'Dude Perfect', 'Movieclips', 'Tips Official',
       'El Reino Infantil', 'Wave Music', 'Aaj Tak', 'Sony Music India',
       'EminemMusic', 'Marshmello', 'YRF', 'Ed Sheeran',
       'Infobells - Hindi', 'Taylor Swift', 'Ariana Grande',
       'BillionSurpriseToys  - Nursery Rhymes & Cartoons', 'JuegaGerman',
       'Get Movies', 'Shemaroo', 'SonyMusicIndiaVEVO', 'Fernanfloo',
       'Voot Kids', 'Felipe Neto', 'HAR PAL GEO', 'Katy Perry',
       'whinderssonnunes', 'ABS-CBN Entertainment', 'Shakira',
       'ARY Digital HD', 'Speed Records', 'Masha and The Bear',
       'Like Nas

In [34]:
df5.shape

(687, 18)

In [35]:
df5.head()

,Youtuber,subscribers,video views,category,uploads,Country,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Urban_population
0,T-Series,245000000,2.280000e+11,Music,20082,India,2.258000e+09,564600.0,9000000.0,6800000.0,108400000.0,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,471031528.0
2,MrBeast,166000000,2.836884e+10,Entertainment,741,United States,1.348000e+09,337000.0,5400000.0,4000000.0,64700000.0,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,270663028.0
3,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,966,United States,1.975000e+09,493800.0,7900000.0,5900000.0,94800000.0,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,270663028.0
4,SET India,159000000,1.480000e+11,Shows,116536,India,1.824000e+09,455900.0,7300000.0,5500000.0,87500000.0,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,471031528.0
7,PewDiePie,111000000,2.905804e+10,Gaming,4716,Japan,3.918400e+07,9800.0,156700.0,117600.0,1900000.0,0.0,2010.0,Apr,29.0,63.2,1.262266e+08,115782416.0


In [36]:
df5.describe()

,subscribers,video views,uploads,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_date,Gross tertiary education enrollment (%),Population,Urban_population
count,6.870000e+02,6.870000e+02,687.000000,6.870000e+02,687.000000,6.870000e+02,6.870000e+02,6.870000e+02,6.870000e+02,687.000000,687.000000,687.000000,6.870000e+02,6.870000e+02
mean,2.317744e+07,1.171128e+10,12276.350801,1.839032e+08,42940.105284,6.866772e+05,5.148103e+05,8.244146e+06,2.701404e+05,2012.294032,15.724891,62.474818,4.794712e+08,2.401945e+08
std,1.818424e+07,1.583346e+10,39682.308851,3.685634e+08,72449.374452,1.158135e+06,8.681193e+05,1.391045e+07,5.712630e+05,4.423401,8.765407,26.434859,4.971390e+08,1.558879e+08
min,1.230000e+07,2.056338e+07,104.000000,3.360000e+02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1970.000000,1.000000,9.000000,1.912789e+06,1.304943e+06
25%,1.450000e+07,4.330698e+09,459.500000,2.843650e+07,6700.000000,1.073000e+05,8.045000e+04,1.300000e+06,0.000000e+00,2009.000000,8.000000,35.500000,1.260140e+08,7.992776e+07
50%,1.770000e+07,7.741765e+09,1189.000000,7.940200e+07,18900.000000,3.031000e+05,2.273000e+05,3.600000e+06,1.000000e+05,2013.000000,16.000000,61.900000,3.282395e+08,2.706630e+08
75%,2.435000e+07,1.418384e+10,3921.500000,1.827575e+08,45000.000000,7.198500e+05,5.399000e+05,8.600000e+06,3.000000e+05,2015.000000,23.000000,88.200000,3.282395e+08,2.706630e+08
max,2.450000e+08,2.280000e+11,301308.000000,6.148000e+09,576000.000000,9.200000e+06,6.900000e+06,1.106000e+08,8.000000e+06,2022.000000,31.000000,113.100000,1.366418e+09,4.710315e+08


## Export the final dataframe to a new csv file

In [39]:
with open('clean_data.csv', 'w') as file:
    file.write(df5.to_csv(index=False, lineterminator='\n'))